In [1]:
import emoji
import pandas as pd
import numpy as np
from keras.models import Model, load_model

Using TensorFlow backend.


In [2]:
emoji_dictionary = {'0': '\u2764\uFE0F',
                    '1': ':baseball:',
                    '2': ':grinning_face_with_big_eyes:',
                    '3': ':disappointed_face:',
                    '4': ':fork_and_knife:',
                    '5': ':hundred_points:',
                    '6': ':fire:',
                    '7': ':face_blowing_a_kiss:',
                    '8': ':chestnut:',
                    '9': ':flexed_biceps:'}

## Exporting model to AI Platform

In [3]:
model = load_model('best_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [4]:
from sklearn.externals import joblib
joblib.dump(model, 'model.joblib')

['model.joblib']

In [6]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate('service-account.json')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

print('initialized')

initialized


In [7]:
from firebase_admin import storage
bucket = storage.bucket(name='machine-learning-in-the-cloud.appspot.com')

In [8]:
b = bucket.blob('emoji-v1/model.joblib')
b.upload_from_filename('model.joblib')
print('model uploaded!')

model uploaded!


In [16]:
f = open('glove.6B.50d.txt', encoding='utf-8')
embeddings_index = {}

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float')
    embeddings_index[word] = coefs
    
f.close()

emb_dim = embeddings_index['eat'].shape[0]

In [17]:
def embedding_output(X):
    maxLen = 10
    embedding_out = np.zeros((X.shape[0], maxLen, emb_dim))
    
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for ij in range(len(X[ix])):
            try:
                embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij] = np.zeros((50,))
    
    return embedding_out

In [18]:
def predict_emoji(sentence):
    pred = model.predict_classes(embedding_output(pd.Series([sentence])))
    print(sentence)
    print(emoji.emojize(emoji_dictionary[str(pred[0])]))